#### 필요 패키지

In [7]:
# !pip install kiwipiepy
# !pip install --upgrade cohere
# !pip install cohere openai langchain pinecone-client
# !pip install dill
# !pip install rank-bm25
# !pip install rank-bm25 nltk

#### bm25_retriever_ooo.pkl 파일 만드는 코드 

In [10]:
import json
import pickle
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# NLTK tokenizer를 사용하기 위한 다운로드 (최초 1회 필요)
nltk.download('punkt')

# 🔹 JSON 파일 경로
json_file_path = "../parse/sony_zv-1_list.json"
bm25_pkl_file_path = "../data/bm25_retrieve_zv-1.pkl"

# 🔹 JSON 파일 로드
with open(json_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# 🔹 BM25 인덱스를 위한 문서 리스트 및 ID 저장
documents = []
document_ids = []

for page in data:
    page_number = page.get("page", "unknown")
    for item in page.get("items", []):
        if "md" in item:  # Markdown 텍스트 추출
            text_content = item["md"]
            documents.append(text_content)
            document_ids.append(f"page-{page_number}-{item.get('value', 'unknown')}")

# 🔹 문서 토큰화
tokenized_corpus = [word_tokenize(doc.lower()) for doc in documents]

# 🔹 BM25 모델 학습
bm25 = BM25Okapi(tokenized_corpus)

# 🔹 BM25 모델 및 문서 ID 저장
with open(bm25_pkl_file_path, "wb") as pkl_file:
    pickle.dump((bm25, document_ids, documents), pkl_file)

print(f"✅ BM25 모델이 포함된 `bm25_retrieve_zv-1.pkl` 파일 생성 완료: {bm25_pkl_file_path}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ZeroDivisionError: division by zero

In [8]:
import json
import pickle
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# NLTK 토크나이저 다운로드 (최초 실행 시 필요)
nltk.download('punkt')

class BM25Retriever:
    def __init__(self, json_path=None, pkl_path=None):
        """
        JSON 파일에서 데이터를 로드하여 BM25 모델을 생성하거나,
        기존 BM25 모델을 pkl 파일에서 로드하여 사용.
        """
        if pkl_path:
            self.load_from_pkl(pkl_path)
        elif json_path:
            self.load_from_json(json_path)
        else:
            raise ValueError("JSON 또는 PKL 파일 경로를 제공해야 합니다.")

    def load_from_json(self, json_path):
        """JSON 파일에서 데이터를 로드하여 BM25 모델을 생성"""
        with open(json_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)

        self.documents = []
        self.document_ids = []
        
        for page in data:
            page_number = page.get("page", "unknown")
            for item in page.get("items", []):
                if "md" in item:
                    text_content = item["md"]
                    self.documents.append(text_content)
                    self.document_ids.append(f"page-{page_number}-{item.get('value', 'unknown')}")

        self.tokenized_corpus = [word_tokenize(doc.lower()) for doc in self.documents]
        self.bm25 = BM25Okapi(self.tokenized_corpus)

    def save_to_pkl(self, pkl_path):
        """BM25 모델을 pkl 파일로 저장"""
        with open(pkl_path, "wb") as pkl_file:
            pickle.dump((self.bm25, self.document_ids, self.documents), pkl_file)
        print(f"✅ BM25 모델이 `{pkl_path}`에 저장되었습니다.")

    def load_from_pkl(self, pkl_path):
        """BM25 모델을 pkl 파일에서 로드"""
        with open(pkl_path, "rb") as pkl_file:
            self.bm25, self.document_ids, self.documents = pickle.load(pkl_file)

    def retrieve(self, query, top_n=5):
        """주어진 쿼리에 대해 BM25 검색 수행"""
        tokenized_query = word_tokenize(query.lower())
        scores = self.bm25.get_scores(tokenized_query)

        # 상위 n개 검색 결과 가져오기
        top_indexes = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]

        results = []
        for idx in top_indexes:
            results.append({
                "document_id": self.document_ids[idx],
                "text": self.documents[idx],
                "score": scores[idx]
            })

        return results


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
json_path = "../parse/sony_zv-1_list.json"
bm25_pkl_path = "../data/bm25_retrieve_zv-1.pkl"

# BM25 모델 생성 및 저장
retriever = BM25Retriever(json_path=json_path)
retriever.save_to_pkl(bm25_pkl_path)


In [ ]:
import json
import pickle
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# NLTK tokenizer를 사용하기 위한 다운로드 (최초 1회 필요)
nltk.download('punkt')

# JSON 파일 경로
json_file_path = "../parse/sony_zv-1_list.json"
bm25_pkl_file_path = "../data/bm25_retrieve_zv-1.pkl"

# JSON 파일 로드
with open(json_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# 문서 리스트 생성 (BM25 입력용)
documents = []
document_ids = []

for page in data:
    page_number = page.get("page", "unknown")
    for item in page.get("items", []):
        if "md" in item:
            text_content = item["md"]
            documents.append(text_content)
            document_ids.append(f"page-{page_number}-{item.get('value', 'unknown')}")

# 문서 토큰화
tokenized_corpus = [word_tokenize(doc.lower()) for doc in documents]

# BM25 모델 학습
bm25 = BM25Okapi(tokenized_corpus)

# BM25 모델과 문서 ID를 pkl 파일로 저장
with open(bm25_pkl_file_path, "wb") as pkl_file:
    pickle.dump((bm25, document_ids, documents), pkl_file)

print(f"✅ BM25 모델이 포함된 `bm25_retrieve_zv-1.pkl` 파일 생성 완료: {bm25_pkl_file_path}")


#### .pkl파일 만드는 코드

In [2]:
import json
import pickle

# JSON 파일 경로
json_file_path = "../parse/sony_zv-1_list.json"
pkl_file_path = "../data/zv-1.pkl"

# JSON 파일 로드
with open(json_file_path, "r", encoding="utf-8") as json_file: 
    data = json.load(json_file)

# PKL 파일로 저장
with open(pkl_file_path, "wb") as pkl_file:
    pickle.dump(data, pkl_file)

print(f"✅ JSON 데이터를 PKL 파일로 저장 완료: {pkl_file_path}")


✅ JSON 데이터를 PKL 파일로 저장 완료: ../data/zv-1.pkl


In [ ]:
import json
from langchain.retrievers import BM25Retriever
from kiwipiepy import Kiwi
import dill

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

with open("../parse/sony_zv-1_list.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

documents = []
for item in json_data:
    text = item["text_field"]  # "text_field"는 텍스트 데이터를 포함하는 필드 이름
    documents.append(text)

bm25_retriever = BM25Retriever.from_texts(documents, preprocess_func=kiwi_tokenize)

with open("../data/bm25_retriever_a6400.pkl", "wb") as f:
    dill.dump(bm25_retriever, f)

print("bm25_retriever_a6400.pkl 파일 생성 완료")

#### RAG 찐 시작

##### bm25 pkl 파일 생성

In [12]:
import json
import pickle
import dill  # dill은 pickle보다 확장된 직렬화 지원
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

# NLTK 토크나이저 사용을 위한 다운로드 (최초 1회 필요)
nltk.download('punkt')

# 🔹 JSON 파일 경로
json_file_path = "../parse/sony_zv-1_list.json"
bm25_pkl_file_path = "../data/retriever_zv-1.pkl"

# 🔹 JSON 파일 로드
with open(json_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# 🔹 BM25 학습용 문서 리스트 및 ID 저장
documents = []
document_ids = []

for page in data:
    page_number = page.get("page", "unknown")
    for item in page.get("items", []):
        if "md" in item and item["md"].strip():  # 빈 문서 필터링
            text_content = item["md"]
            documents.append(text_content)
            document_ids.append(f"page-{page_number}-{item.get('value', 'unknown')}")

# 🔹 문서 토큰화 (BM25 입력용)
tokenized_corpus = [word_tokenize(doc.lower()) for doc in documents]

# 🔹 BM25 모델 학습
bm25 = BM25Okapi(tokenized_corpus)

# 🔹 BM25 Retriever 데이터 저장
bm25_retriever = {
    "bm25": bm25,
    "document_ids": document_ids,
    "documents": documents
}

with open(bm25_pkl_file_path, "wb") as pkl_file:
    dill.dump(bm25_retriever, pkl_file)

print(f"✅ BM25 Retriever 모델이 `{bm25_pkl_file_path}` 파일로 저장 완료!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'list' object has no attribute 'get'

In [11]:
import getpass
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings  # Correct importo
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from kiwipiepy import Kiwi
from pinecone import Pinecone
import cohere
import dill  # dill import 추가

load_dotenv(override=True)  # 강제 다시 로드

# API 키 확인 및 설정 (기존 코드 유지)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]
cohere_api = os.environ.get("COHERE_API_KEY")  # .get()을 사용하여 키가 없을 때 None 반환

if not cohere_api:
    raise ValueError("COHERE_API_KEY 환경 변수를 설정해주세요.")


# 벡터 스토어 로드 (기존 코드 유지)
pc = Pinecone(api_key=pinecone_api)
index_name = "sony"
index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # Corrected model name

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

retriever = vector_store.as_retriever(
    search_type="similarity", search_kwargs={"k": 10},
)

# Kiwi 토크나이저 (기존 코드 유지)
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# BM25 검색기 로드 (수정: 파일에서 로드)
try:
    with open("../data/zv-1.pkl", "rb") as f:
        bm25_retriever = dill.load(f)
    bm25_retriever.preprocess_func = kiwi_tokenize
except FileNotFoundError:
    print("Warning: zv-1.pkl 파일을 찾을 수 없습니다. BM25 검색을 사용할 수 없습니다.")
    bm25_retriever = None  # BM25 검색기 사용 안함
except Exception as e:  # 다른 예외 처리
    print(f"Error loading BM25 retriever: {e}")
    bm25_retriever = None 

# 앙상블 검색기 생성 (수정: BM25 검색기 존재 여부 확인)
if bm25_retriever:  # BM25 검색기가 로드된 경우에만 앙상블 검색기 생성
    ensemble_retriever = EnsembleRetriever(
        retrievers=[retriever, bm25_retriever],
        weights=[0.5, 0.5]  # Dense와 BM25 각각 50% 가중치
    )
else:
    ensemble_retriever = retriever  # BM25 검색기 없을 경우 벡터 검색기만 사용
    print("Warning: BM25 검색기를 사용할 수 없어, 벡터 검색기만 사용합니다.")


cohere_client = cohere.Client(cohere_api)  # 기존 코드 유지

# ... (이후 검색 및 처리 코드)

Error loading BM25 retriever: 'list' object has no attribute 'preprocess_func'


In [ ]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from kiwipiepy import Kiwi
import cohere
# import dill

load_dotenv(override=True) # 강제 다시 로드

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
  os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]
cohere_api = os.environ["COHERE_API_KEY"]

# vectorstore load
# 문서 검색 (Hybrid Search: BM25 + Pinecone)
# pinecone 벡터 기반 검색


pc = Pinecone(api_key=pinecone_api)

index_name = "sony"
index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)


retriever = vector_store.as_retriever(
  search_type="similarity", search_kwargs={"k": 10},
)

# BM25 키워드 기반 검색 
# ✅ BM25Retriever를 사용하여 키워드 기반 검색 수행
# ✅ 한국어 형태소 분석기 kiwipiepy를 사용하여 검색 성능 최적화
# ✅ BM25 모델을 .pkl 파일로 저장하여 로드 가능 -> drill로 로드
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]



# === 1. BM25Retriever와 Kiwi 로드 ===
with open("../data/zv-1.pkl", "rb") as f:
    bm25_retriever = dill.load(f)


bm25_retriever.preprocess_func = kiwi_tokenize

# === 3. Ensemble Retriever 생성 ===
# BM25 + Pinecone 결합 (Ensemble Search)
# 두 가지 검색 방식을 결합하여 검색 성능 향상 (하이브리드 검색)
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[0.5, 0.5]  # Dense와 BM25 각각 50% 가중치
)

cohere_client = cohere.Client(cohere_api)

#### state

In [ ]:
# #✅ 사용자의 질문과 검색 & 응답 상태를 저장하는 역할
# #✅ 각 단계에서 필요한 데이터를 유지하며 전달됨
# # state 코드 정의 

In [27]:
from typing import Annotated, TypedDict, List

# 1. 질문 관련 State
class QuestionState(TypedDict):
    question: Annotated[str, "Question"]
    transform_question: Annotated[List[str], "Transformed queries generated by LLM"]  # List로 변경

# 2. 검색 및 컨텍스트 관련 State
class ContextRetrievalState(TypedDict):
    ensemble_context: Annotated[str, "Ensemble Retrieve"]
    multi_context: Annotated[str, "Multi Query"]
    merge_context: Annotated[str, "Merge Context"]
    filtered_context: Annotated[str, "Filtering Context"]
    rerank_context: Annotated[str, "Reranked Context"]  # reranked context 혹은 context로 표기하기

# 3. 답변 및 메시지 관련 State
class AnswerState(TypedDict):
    answer: Annotated[str, "Answer"]
    message: Annotated[List[dict], "Messages"] # add_messages type hint 제거. List[dict]로 명시


# 4. 전체 Graph State (각 부분 State를 포함)
class GraphState(TypedDict): 
    question_state: QuestionState
    context_retrieval_state: ContextRetrievalState
    answer_state: AnswerState


# # 사용 예시:
# initial_state: GraphState = {
#     "question_state": {
#         "question": "What is the capital of France?",
#         "transform_question": []  # 초기 빈 리스트
#     },
#     "context_retrieval_state": {
#         "ensemble_context": "",
#         "multi_context": "",
#         "merge_context": "",
#         "filtered_context": "",
#         "rerank_context": ""
#     },
#     "answer_state": {
#         "answer": "",
#         "message": [] # 초기 빈 리스트
#     }
# }


# # 상태 업데이트 예시:
# updated_state: GraphState = initial_state.copy() # 중요: copy()를 사용하여 원본 상태를 변경하지 않도록 함
# updated_state["question_state"]["transform_question"].append("What is the capital of France?")
# updated_state["context_retrieval_state"]["ensemble_context"] = "Paris is the capital..."
# updated_state["answer_state"]["answer"] = "Paris"
# updated_state["answer_state"]["message"].append({"role": "assistant", "content": "Paris is the capital of France."})

# print(updated_state)


# # add_message 함수를 사용하는 예시 (별도 함수로 정의)
# def add_message(state: GraphState, role: str, content: str):
#     new_message = {"role": role, "content": content}
#     updated_state = state.copy()
#     updated_state["answer_state"]["message"].append(new_message)
#     return updated_state

# updated_state = add_message(updated_state, "user", "Thank you!")
# print(updated_state)

#### retrieve

In [ ]:
from typing import Annotated, TypedDict, List
import retriever
import ensemble_retriever
import bm25_retriever

# 🔹 질문 관련 State
class QuestionState(TypedDict):
    question: Annotated[str, "Original question from user"]
    transform_question: Annotated[List[str], "Transformed queries generated by LLM"]  # List 타입 유지

# 🔹 검색 및 컨텍스트 관련 State
class ContextRetrievalState(TypedDict):
    ensemble_context: Annotated[str, "Ensemble Retrieved Context"]
    multi_context: Annotated[str, "Multi Query Retrieved Context"]
    merge_context: Annotated[str, "Merged Context"]
    filtered_context: Annotated[str, "Filtered Context"]
    rerank_context: Annotated[str, "Reranked Context"]  # 표준화된 key 이름 유지

# 🔹 답변 및 메시지 관련 State
class AnswerState(TypedDict):
    answer: Annotated[str, "Final Answer"]
    message: Annotated[List[dict], "Message history"]  # List[dict]로 변경

# 🔹 전체 Graph State (각 State 포함)
class GraphState(TypedDict):
    question_state: QuestionState
    context_retrieval_state: ContextRetrievalState
    answer_state: AnswerState


# 🔹 **기본 Retriever를 이용하여 문서 검색**
def retrieve_document(state: GraphState) -> GraphState:
    """
    기본 Retriever를 사용하여 질문에 대한 문서를 검색합니다.
    """
    question = state["question_state"]["question"]  # 질문 가져오기
    documents = retriever.invoke(question)  # 기본 retriever 호출
    print(f"🔎 Retrieved documents: {documents}")

    # 업데이트된 State 반환
    state["context_retrieval_state"]["multi_context"] = documents  # multi_query에서 가져온 컨텍스트
    return state


# 🔹 **Ensemble Retriever를 이용한 검색**
def ensemble_document(state: GraphState) -> GraphState:
    """
    Ensemble Retriever를 사용하여 질문에 대한 문서를 검색합니다.
    """
    question = state["question_state"]["question"]  # 질문 가져오기
    documents = ensemble_retriever.invoke(question)  # Ensemble retriever 호출
    print(f"🔎 Ensemble Retrieved documents: {documents}")

    # 업데이트된 State 반환
    state["context_retrieval_state"]["ensemble_context"] = documents  # Ensemble retrieval 적용 결과 저장
    return state


# 🔹 **BM25 Retriever를 이용한 검색 (옵션)**
def bm25_document(state: GraphState) -> GraphState:
    """
    BM25 기반 검색을 수행합니다.
    """
    question = state["question_state"]["question"]  # 질문 가져오기
    if bm25_retriever:
        documents = bm25_retriever.invoke(question)  # BM25 retriever 호출
        print(f"🔎 BM25 Retrieved documents: {documents}")

        # 업데이트된 State 반환
        state["context_retrieval_state"]["filtered_context"] = documents  # BM25 기반 컨텍스트 저장
    else:
        print("⚠️ BM25 Retriever가 존재하지 않습니다. 기본 검색만 수행됩니다.")

    return state


In [30]:
# import ContextRetrievalState
# import retriever, ensemble_retriever, bm25_retriever

# def retrieve_document(state: GraphState) -> GraphState:
#     questions = state["question"]
#     documents = retriever.invoke(questions)
#     print(documents)
#     return {"context": documents, "question": questions}

# # Ensemble retriever 정의
# def ensemble_document(state: GraphState) -> GraphState:
#     questions = state["question"]
#     documents = ensemble_retriever.invoke(questions)
#     # print(documents)
#     return {"ensemble_context": documents}


#### multiquery

#### pinecone

In [ ]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Pinecone
from langchain.retrievers import BM25Retriever
from langchain.embeddings.openai import OpenAIEmbeddings

# 환경 변수 로드 (API 키를 .env 파일에 저장하고 로드합니다.)
load_dotenv()

# OpenAI API 키 확인 및 설정
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수를 설정해주세요.")

# OpenAI Embeddings 초기화 (API 키 포함)
embed_model = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-ada-002") # 모델 지정 (선택적)

# Pinecone 벡터 스토어 초기화 (index 객체가 미리 정의되어 있어야 합니다.)
pinecone_retriever = Pinecone(index, embed_model)

# BM25 키워드 검색기 초기화 (documents 리스트가 미리 정의되어 있어야 합니다.)
bm25_retriever = BM25Retriever.from_texts(documents)

# Hybrid 검색 (BM25 + Pinecone)
query = "Sony 카메라의 크기"
bm25_results = bm25_retriever.get_relevant_documents(query)
pinecone_results = pinecone_retriever.similarity_search(query, top_k=5)

# 두 결과를 결합하여 Hybrid 검색 결과 생성 (중복 제거)
combined_results = list(set(bm25_results + pinecone_results))  # 중복 제거 및 리스트로 변환

# 출력
print(" Hybrid 검색 결과:") 
for doc in combined_results:
    print(f"- {doc.page_content}")

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.retrievers import BM25Retriever
# from langchain.vectorstores import BM25Retriever
from langchain.embeddings.openai import OpenAIEmbeddings



# 🔹 Pinecone 벡터 스토어 초기화
pinecone_retriever = Pinecone(index, embed_model)

# 🔹 BM25 키워드 검색기 초기화
bm25_retriever = BM25Retriever.from_texts(documents)

# 🔹 Hybrid 검색 (BM25 + Pinecone)
query = "Sony 카메라의 크기"
bm25_results = bm25_retriever.get_relevant_documents(query)
pinecone_results = pinecone_retriever.similarity_search(query, top_k=5)

# 🔹 두 결과를 결합하여 Hybrid 검색 결과 생성
combined_results = bm25_results + pinecone_results

# 🔹 출력
print("🔍 Hybrid 검색 결과:")
for doc in combined_results:
    print(f"- {doc.page_content}")


#### cohere reranker

In [32]:

# from state import GraphState
# from ingestion import cohere_client


def rerank_with_cohere(query, retrieved_docs, top_n=5):
    # Cohere에 전달할 문서 형식
    documents = [doc.page_content for doc in retrieved_docs]
    
    # Reranker 호출a
    
    response = cohere_client.rerank(
        query=query,
        documents=documents,
        top_n=top_n,  # 상위 N개 문서 선택
        model="rerank-v3.5"  # 사용할 Cohere Reranker 모델
    )
    
    # 상위 문서만 반환
    reranked_docs = [retrieved_docs[result.index] for result in response.results]
    return reranked_docs

# Reranker Node
def rerank_docs(state: GraphState) -> GraphState:
    
    questions = state['question']
    documents = state['merge_context']
    reranked_docs = rerank_with_cohere(questions, documents)
    return {"rerank_context": reranked_docs}

#### generation

In [34]:
# generation.py
# 답변 생성 체인
from dotenv import load_dotenv
# from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = ChatOpenAI(temperature=0, model="gpt-4o")
# prompt = hub.pull("rlm/rag-prompt")

ANSWER_PROMPT = PromptTemplate(
    input_variables=["question","context"],
    template="""
당신은 카메라 사용자 메뉴얼에 대한 정보를 제공하는 AI 어시스턴트입니다. 사용자가 질문을 하면, 제공된 Document 형식의 context를 활용하여 답변을 생성하세요. 각 Document에는 이미지 경로가 포함된 metadata가 있습니다. 답변을 생성할 때, 관련된 이미지가 있는 경우 [image: metadata 내 이미지 경로] 형식으로 답변에 포함시켜 주세요. 

예시:
사용자 질문: "카메라의 ISO 설정 방법을 알려주세요."
답변: "카메라의 ISO 설정은 메뉴에서 '설정'을 선택한 후 'ISO' 옵션을 선택하여 조정할 수 있습니다. [image: /path/to/iso_setting_image]"

이와 같은 형식으로 질문에 대한 답변을 생성하세요.

컨텍스트와 질문을 기반으로 답변을 생성하세요:
- 컨텍스트: {context}
- 질문: {question}
"""
)

generation_chain = ANSWER_PROMPT | llm | StrOutputParser()

### my RAG

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # 메시지 정의(list type 이며 add_messages 함수를 사용하여 메시지를 추가)
    messages: Annotated[list, add_messages]


In [ ]:
from langchain_openai import ChatOpenAI

# LLM 정의
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm.invoke(state["messages"])]}


In [ ]:
# 그래프 생성
graph_builder = StateGraph(State)

# 노드 이름, 함수 혹은 callable 객체를 인자로 받아 노드를 추가
graph_builder.add_node("chatbot", chatbot)


In [ ]:
# 시작 노드에서 챗봇 노드로의 엣지 추가
graph_builder.add_edge(START, "chatbot")

In [ ]:
# 그래프에 엣지 추가
graph_builder.add_edge("chatbot", END)

In [ ]:
# 그래프 컴파일
graph = graph_builder.compile()

In [ ]:
question = "카메라 IOS값 설정법 알려줘줘"

# 그래프 이벤트 스트리밍
for event in graph.stream({"messages": [("user", question)]}):
    # 이벤트 값 출력
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)
